# Data Collection: 

## 4. Scraping Wikipedia Artists and Songs

- This script will iterate through all songs and artists in the billboard year-end top 100 list and scrape the following:
    - gender of the artist
    - age of the artist
    - song genre
    - artist genre
    - group: whether the artist is a single person of a group act
    - colab: whether the song is performed by a single artist or a group of artists

## Get_Info Pipeline psuedo code:
- This function is for scraping one single wikipedia page
- if type is song:
    - scrape box: get genre
- if type is artist:
    - scrape box:
        - get genre
        - if box has Members --> Group Artist
            - Iterate, scrape each member's gender and age
            - Average members' age for artist age
            - Determine if group is all male, all female, or mixed
        - otherwise --> Single Artist
            - get age (year - born)
            - scrape text: get gender using gender pronouns

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
from time import sleep

In [7]:
def get_info(url, type_, releaseyear):
    '''
    get wikipedia info box for one artist
    input:
        url: string, link of wikipedia page
        type of page: string, 'artist' or 'song'
        release year: int
    output:
        dictionary object
    '''
    # get wikipedia page
    response = requests.get(url)
    soup = BeautifulSoup(response.text, parser = 'html.parser')
    print(url)
    
    # There are three types of table objects, iterate until it find the appropriate info box
    table = soup.find('table', class_ = 'infobox biography vcard')
    if not table:
        table = soup.find('table', class_= 'infobox vcard plainlist')
    if not table:
        table = soup.find('table', class_ = 'infobox vevent')
    if not table:
        table = soup.find('table', class_ = 'infobox vcard')
    # if there are no table, get gender (which only requires first paragraph) and return
    if not table:
        try:
            return {'gender': get_gender(soup, 0.7)}
        except:
            return 'unscrapable'
    
    # get all info in the table
    labels = table.find_all('th', class_ = 'infobox-label')
    data = table.find_all('td', class_ = 'infobox-data')
    lablst = [label.text for label in labels]
    datalst = [d.text.replace('\n', ' ').strip() for d in data]

    # make dictionary
    dict_ = {}
    for data, lab in zip(datalst, lablst):
        dict_[lab] = data
    
    ## get specific elements and add to dictionary
    # get genre as list
    if 'Genre' in dict_ or 'Genres' in dict_:
        try:
            sibling = table.find('th', class_ = 'infobox-label', text = 'Genres').find_next_sibling()
        except:
            sibling = table.find('th', class_ = 'infobox-label', text = 'Genre').find_next_sibling()
        genres = [genre.text for genre in sibling.find_all('a')]    
        dict_['Genre'] = genres
    
    # get members as list
    if 'Members' in dict_:
        sibling = table.find('th', class_ = 'infobox-label', text = 'Members').find_next_sibling()
        members = [member.text for member in sibling.find_all('a')]
        if members:
            dict_['Members_lst'] = members
        else:
            dict_['Members_lst'] = sibling.text
            
    elif 'Past members' in dict_:
        sibling = table.find('th', class_ = 'infobox-label', text = 'Past members').find_next_sibling()
        members = [member.text for member in sibling.find_all('a')]
        if members:
            dict_['Members_lst'] = members
        else:
            dict_['Members_lst'] = sibling.text
        
    # get age for individuals, get average age for group
        # if song is a collaboration, the average age will be calculated in the add_info function
    # get gender for individuals and groups,
        # single-gender artists and groups are labeled as 'male' or 'female',
        # mixed-gender groups will be labeled 'mixed'
        # mixed-gender collaboration will be labeled in the add_info function
    if type_ == 'song':
        pass
    # individual
    elif 'Born' in dict_:
        birthyear = re.findall(r'\d\d\d\d', dict_['Born'])
        if birthyear:
            dict_['age'] = releaseyear - int(birthyear[0])
        dict_['gender'] = get_gender(soup, 0.7)
    # group
    elif 'Members_lst' in dict_:
        age = []
        gender = []
        # if hyperlinks exist for the members get them, otherwise label as 'unknown'
        if type(dict_['Members_lst']) == list:
            for member in dict_['Members_lst']:
                # use find element by text to get url
                link = soup.find('a', text = member)['href']
                link = 'https://en.wikipedia.org/' + link
                artist_dict = get_info(link, 'artist', releaseyear)
                if 'age' in artist_dict:
                    age.append(artist_dict['age'])
                if 'gender' in artist_dict:
                    gender.append(artist_dict['gender'])
            dict_['age'] = np.array(age).mean()

            if 'male' in gender and 'female' in gender:
                dict_['gender'] = 'mixed'
            elif 'male' not in gender and 'female' not in gender:
                dict_['gender'] = 'unknown'
            elif 'male' in gender:
                dict_['gender'] = 'male'
            else:
                dict_['gender'] = 'female'

    sleep(3)
    return dict_


In [4]:
def get_gender(soup, ratio):
    '''
    get gender of artist using first wikipedia paragraph
    Input:
        soup: soup object for the page
        ratio: ratio of gender pronouns to decide whether a person is male or female
    '''
    # get text
    text = soup.text
    # tokenize and count gender pronouns
    tokens = text.split()
    tokens = [token.lower() for token in tokens]
    male = tokens.count('he') + tokens.count('him') + tokens.count('his')
    female =  tokens.count('she') + tokens.count('her') + tokens.count('hers')
    # if no gender pronouns, verdict is unknown, otherwise count percentage
    if male + female == 0:
        verdict = 'unknown'
    elif male / (male + female) >= ratio:
        verdict = 'male'
    elif female / (male + female) >= ratio:
        verdict = 'female'
    # if no single gender pronoun crosses threshold, verdict is also unknown
    else:
         verdict = 'unknown'
    return verdict

## Example:
- Single Artist: Dua Lipa
- Group Artist: BTS
- Song: Levitating

In [10]:
d1 = get_info('https://en.wikipedia.org/wiki/Dua_Lipa', 'artist', 2021)
d1

https://en.wikipedia.org/wiki/Dua_Lipa


{'Born': '(1995-08-22) 22 August 1995 (age\xa026)London, England',
 'Occupation': 'Singersongwriter',
 'Years\xa0active': '2014–present',
 'Awards': 'Full list',
 'Genres': 'Pop disco house R&B',
 'Instruments': 'Vocals',
 'Labels': 'Warner Urban',
 'Associated acts': 'The Blessed Madonna',
 'Website': 'dualipa.com',
 'Genre': ['Pop', 'disco', 'house', 'R&B'],
 'age': 26,
 'gender': 'female'}

In [11]:
d2 = get_info('https://en.wikipedia.org/wiki/Levitating_(song)', 'song', 2021)
d2

https://en.wikipedia.org/wiki/Levitating_(song)


{'Released': '1\xa0October\xa02020\xa0(2020-10-01)',
 'Recorded': '28\xa0August\xa02018\xa0(2018-08-28)',
 'Studio': 'Sarm (London, United Kingdom) the Bunker at 13 Sarm Music Village (London, United Kingdom) Modulator Music (Toronto, Canada)',
 'Genre': ['Electro', 'disco', 'nu-disco', 'pop-funk', 'space rock', '[1]'],
 'Length': '3:23',
 'Label': 'Warner',
 'Songwriter(s)': 'Dua Lipa Clarence Coffee Jr. Sarah Hudson Stephen Kozmeniuk DaBaby[a]',
 'Producer(s)': 'Stephen Kozmeniuk Stuart Price'}

In [12]:
d3 = get_info('https://en.wikipedia.org/wiki/BTS', 'artist', 2021)
d3

https://en.wikipedia.org/wiki/BTS
https://en.wikipedia.org//wiki/Kim_Seok-jin
https://en.wikipedia.org//wiki/Suga_(rapper)
https://en.wikipedia.org//wiki/J-Hope
https://en.wikipedia.org//wiki/RM_(rapper)
https://en.wikipedia.org//wiki/Jimin_(singer,_born_1995)
https://en.wikipedia.org//wiki/V_(singer)
https://en.wikipedia.org//wiki/Jungkook


{'Also known as': 'Bangtan Boys Bangtan Sonyeondan Beyond the Scene Bulletproof Boy Scouts',
 'Origin': 'Seoul, South Korea',
 'Genres': 'K-pop pop hip hop R&B EDM',
 'Years active': '2013–present',
 'Labels': 'Big Hit Pony Canyon Def Jam Japan[1] Columbia[2][3] Universal[4]',
 'Associated acts': 'Steve Aoki',
 'Website': 'bts.ibighit.com',
 'Members': 'Jin Suga J-Hope RM Jimin V Jungkook',
 'Hangul': '방탄소년단',
 'Hanja': '防彈少年團',
 'Revised Romanization': 'Bangtan Sonyeondan',
 'McCune–Reischauer': "Pangt'an Sonyǒndan",
 'Kanji': '防弾少年団',
 'Hiragana': 'ぼうだんしょうねんだん',
 'Revised Hepburn': 'Bōdan Shōnendan',
 'Kunrei-shiki': 'Bôdan Syônendan',
 'Genre': ['K-pop', 'pop', 'hip hop', 'R&B', 'EDM'],
 'Members_lst': ['Jin', 'Suga', 'J-Hope', 'RM', 'Jimin', 'V', 'Jungkook'],
 'age': 26.714285714285715,
 'gender': 'male'}

# Scraping Preparation
- Open DataFrame of Song and Artist Links
- add_info_row function to write scraped results into dataframe
- missing list to keep track of missing values in original DataFrame
    - this will help you determine whether there's too much data missing

## add_info_row Psuedo code
- For each row: create song_dict and artist_dict_lst to record information
- Send song url to get_info to get song genre
- For each artist (there may be more than one artist if it's a collaboration):
    - send artist to get_info to get age, gender, artist genre
    - If there are more than one artist:
        - average age
        - determine if gender is all male, all female, mixed (again)
        - get overlapping genre 

In [54]:
with open('../data/top100_1958_to_2021.csv', 'r') as f:
    df = pd.read_csv(f)

In [1]:
def add_info_row(df, index):
    '''
    Given an index and a dataframe with song and artist links, scrape and add data to that index
    Input:
        Dataframe object to add to
        Index of row to scrape
    Output: None
    '''
    # turn string back into list
    song = df.loc[index, 'song_wiki_url'].strip('[]\"\'').split('\', \'')[0]
    artist_lst = df.loc[index, 'artist_wiki_url'].strip('[]\"\'').split('\', \'')
    song_dict = get_info(song, 'song', df.loc[index, 'year'])
    artist_dict_lst = [get_info(artist, 'artist', df.loc[index, 'year']) for artist in artist_lst]
    try:
        df.loc[index, 'song_genre'] = str(song_dict['Genre'])
    except KeyError:
        df.loc[index, 'song_genre'] = 'unknown'
    # get genre for all artist and flatten list, will contain repeats
    try:
        df.loc[index, 'artist_genre'] = str([d['Genre'] for d in artist_dict_lst])
    except KeyError:
        df.loc[index, 'artist_genre'] = 'unknown'
    # get gender
    gender_lst = []
    for artist_dict in artist_dict_lst:
        if 'gender' in artist_dict:
            gender_lst = artist_dict['gender']
    if 'mixed' in gender_lst:
        df.loc[index, 'artist_gender'] = 'mixed'
    elif 'male' in gender_lst and 'female' in gender_lst:
        df.loc[index, 'artist_gender'] = 'mixed'
    elif 'male' not in gender_lst and 'female' not in gender_lst:
        df.loc[index, 'artist_gender'] = 'unknown'
    elif 'male' in gender_lst:
        df.loc[index, 'artist_gender'] = 'male'
    else:
        df.loc[index, 'artist_gender'] = 'female'
    # get average age
    age = []
    if len(artist_dict_lst) == 1:
        df.loc[index, 'collab'] = False
    elif len(artist_dict_lst) > 1:
        df.loc[index, 'collab'] = True
    
    for artist_dict in artist_dict_lst:
        if 'age' in artist_dict:
            age.append(artist_dict['age'])
        if 'Members_lst' in artist_dict:
            df.loc[index, 'type'] = 'group'
        else:
            df.loc[index, 'type'] = 'single'
    df.loc[index, 'artist_age'] = np.array(age).mean()
    return

## Manual Scraping Pipeline
- Initiate missing values list
- Enter starting index
- Monitor progress (code will write df to file every 50 entries)
- Restart if function starts going recursive:
    - eg when the Members wiki redirects to the group wiki
    - https://en.wikipedia.org/wiki/Alvin_and_the_Chipmunks#Alvin
- Check missing values every now and then

In [12]:
missing = []

In [100]:
missing.append(3921)

In [20]:
missing

[4131, 4339, 4438, 4532]

In [25]:
ct = 0
for i in range(5920, 6351):
    print(str(i) + ' / 6351 ' + str(round(i / 6351 * 100, 3)) + '% ' + str(df.loc[i, 'year']) + ' no. ' + str(df.loc[i, 'no.']))
    if df.loc[i, 'song_wiki_url'] == '[]' or df.loc[i, 'artist_wiki_url'] == '[]':
        print('no link for song or artist', df.loc[i, 'song'], df.loc[i, 'artist(s)'])
        missing.append(i)
        continue
    add_info_row(df, i)
    # autosave per 50 entries
    ct += 1
    if ct % 50 == 0:
        df.to_csv('../data/wiki_box_data.csv')

5920 / 6351 93.214% 2017 no. 70
https://en.wikipedia.org/wiki/Broccoli_(D.R.A.M._song)
https://en.wikipedia.org/wiki/DRAM_(rapper)
https://en.wikipedia.org/wiki/Lil_Yachty
5921 / 6351 93.229% 2017 no. 71
https://en.wikipedia.org/wiki/Slide_(Calvin_Harris_song)
https://en.wikipedia.org/wiki/Calvin_Harris
https://en.wikipedia.org/wiki/Frank_Ocean
https://en.wikipedia.org/wiki/Migos
https://en.wikipedia.org//wiki/Quavo
https://en.wikipedia.org//wiki/Offset_(rapper)
https://en.wikipedia.org//wiki/Takeoff_(rapper)
5922 / 6351 93.245% 2017 no. 72
https://en.wikipedia.org/wiki/What_Ifs
https://en.wikipedia.org/wiki/Kane_Brown
https://en.wikipedia.org/wiki/Lauren_Alaina
5923 / 6351 93.261% 2017 no. 73
https://en.wikipedia.org/wiki/Chained_to_the_Rhythm
https://en.wikipedia.org/wiki/Katy_Perry
https://en.wikipedia.org/wiki/Skip_Marley
5924 / 6351 93.277% 2017 no. 74
https://en.wikipedia.org/wiki/Feels_(song)
https://en.wikipedia.org/wiki/Calvin_Harris
https://en.wikipedia.org/wiki/Pharrell_Will

/var/folders/7n/47sl211117v2958gd5nysr5r0000gn/T/ipykernel_87730/247977764.py:49: RuntimeWarning: Mean of empty slice.
  df.loc[index, 'artist_age'] = np.array(age).mean()
/Users/hsinkengling/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


5941 / 6351 93.544% 2017 no. 91
https://en.wikipedia.org/wiki/No_Promises_(Cheat_Codes_song)
https://en.wikipedia.org/wiki/Cheat_Codes_(DJs)
https://en.wikipedia.org//wiki/KEVI
https://en.wikipedia.org/wiki/Demi_Lovato
5942 / 6351 93.56% 2017 no. 92
https://en.wikipedia.org/wiki/Treat_You_Better_(Shawn_Mendes_song)
https://en.wikipedia.org/wiki/Shawn_Mendes
5943 / 6351 93.576% 2017 no. 93
https://en.wikipedia.org/wiki/I_Get_the_Bag
https://en.wikipedia.org/wiki/Gucci_Mane
https://en.wikipedia.org/wiki/Migos
https://en.wikipedia.org//wiki/Quavo
https://en.wikipedia.org//wiki/Offset_(rapper)
https://en.wikipedia.org//wiki/Takeoff_(rapper)
5944 / 6351 93.592% 2017 no. 94
https://en.wikipedia.org/wiki/Small_Town_Boy_(song)
https://en.wikipedia.org/wiki/Dustin_Lynch
5945 / 6351 93.607% 2017 no. 95
https://en.wikipedia.org/wiki/Everyday_We_Lit
https://en.wikipedia.org/wiki/YFN_Lucci
https://en.wikipedia.org/wiki/PnB_Rock
5946 / 6351 93.623% 2017 no. 96
https://en.wikipedia.org/wiki/Havana_(C

https://en.wikipedia.org//wiki/Luke_Hemmings
https://en.wikipedia.org//wiki/Calum_Hood
https://en.wikipedia.org//wiki/Ashton_Irwin
https://en.wikipedia.org//wiki/Michael_Clifford_(musician)
5987 / 6351 94.269% 2018 no. 37
https://en.wikipedia.org/wiki/Whatever_It_Takes_(Imagine_Dragons_song)
https://en.wikipedia.org/wiki/Imagine_Dragons
https://en.wikipedia.org//wiki/Dan_Reynolds
https://en.wikipedia.org//wiki/Wayne_Sermon
https://en.wikipedia.org//wiki/Ben_McKee
https://en.wikipedia.org//wiki/Daniel_Platzman
5988 / 6351 94.284% 2018 no. 38
https://en.wikipedia.org/wiki/Ric_Flair_Drip
https://en.wikipedia.org/wiki/Offset_(rapper)
https://en.wikipedia.org/wiki/Metro_Boomin
5989 / 6351 94.3% 2018 no. 39
https://en.wikipedia.org/wiki/I_Fall_Apart
https://en.wikipedia.org/wiki/Post_Malone
5990 / 6351 94.316% 2018 no. 40
https://en.wikipedia.org/wiki/Pray_for_Me_(The_Weeknd_and_Kendrick_Lamar_song)
https://en.wikipedia.org/wiki/The_Weeknd
https://en.wikipedia.org/wiki/Kendrick_Lamar
5991 / 

https://en.wikipedia.org/wiki/Florida_Georgia_Line
https://en.wikipedia.org//wiki/Tyler_Hubbard
https://en.wikipedia.org//wiki/Brian_Kelley_(musician)
6033 / 6351 94.993% 2018 no. 83
https://en.wikipedia.org/wiki/Lemon_(N.E.R.D_and_Rihanna_song)
https://en.wikipedia.org/wiki/N.E.R.D
https://en.wikipedia.org//wiki/Pharrell_Williams
https://en.wikipedia.org//wiki/Chad_Hugo
https://en.wikipedia.org//wiki/Shay_Haley
https://en.wikipedia.org/wiki/Rihanna
6034 / 6351 95.009% 2018 no. 84
https://en.wikipedia.org/wiki/1-800-273-8255_(song)
https://en.wikipedia.org/wiki/Logic_(musician)
https://en.wikipedia.org/wiki/Alessia_Cara
https://en.wikipedia.org/wiki/Khalid_(singer)
6035 / 6351 95.024% 2018 no. 85
https://en.wikipedia.org/wiki/Say_Something_(Justin_Timberlake_song)
https://en.wikipedia.org/wiki/Justin_Timberlake
https://en.wikipedia.org/wiki/Chris_Stapleton
6036 / 6351 95.04% 2018 no. 86
https://en.wikipedia.org/wiki/I%27m_Upset
https://en.wikipedia.org/wiki/Drake_(musician)
6037 / 6351

...

6133 / 6351 96.567% 2019 no. 83
https://en.wikipedia.org/wiki/How_Do_You_Sleep%3F_(Sam_Smith_song)
https://en.wikipedia.org/wiki/Sam_Smith_(singer)
6134 / 6351 96.583% 2019 no. 84
https://en.wikipedia.org/wiki/Baby_(Quality_Control,_Lil_Baby_and_DaBaby_song)
https://en.wikipedia.org/wiki/Lil_Baby
https://en.wikipedia.org/wiki/DaBaby
6135 / 6351 96.599% 2019 no. 85
https://en.wikipedia.org/wiki/Look_What_God_Gave_Her
https://en.wikipedia.org/wiki/Thomas_Rhett
6136 / 6351 96.615% 2019 no. 86
https://en.wikipedia.org/wiki/Good_as_You
https://en.wikipedia.org/wiki/Kane_Brown
6137 / 6351 96.63% 2019 no. 87
https://en.wikipedia.org/wiki/Clout_(Offset_song)
https://en.wikipedia.org/wiki/Offset_(rapper)
https://en.wikipedia.org/wiki/Cardi_B
6138 / 6351 96.646% 2019 no. 88
https://en.wikipedia.org/wiki/Love_Lies_(song)
https://en.wikipedia.org/wiki/Khalid_(singer)
https://en.wikipedia.org/wiki/Normani
6139 / 6351 96.662% 2019 no. 89
https://en.wikipedia.org/wiki/One_Thing_Right
https://en.wikip

https://en.wikipedia.org/wiki/Drake_(musician)
6187 / 6351 97.418% 2020 no. 37
https://en.wikipedia.org/wiki/My_Oh_My_(Camila_Cabello_song)
https://en.wikipedia.org/wiki/Camila_Cabello
https://en.wikipedia.org/wiki/DaBaby
6188 / 6351 97.433% 2020 no. 38
https://en.wikipedia.org/wiki/Dynamite_(BTS_song)
https://en.wikipedia.org/wiki/BTS
https://en.wikipedia.org//wiki/Kim_Seok-jin
https://en.wikipedia.org//wiki/Suga_(rapper)
https://en.wikipedia.org//wiki/J-Hope
https://en.wikipedia.org//wiki/RM_(rapper)
https://en.wikipedia.org//wiki/Jimin_(singer,_born_1995)
https://en.wikipedia.org//wiki/V_(singer)
https://en.wikipedia.org//wiki/Jungkook
6189 / 6351 97.449% 2020 no. 39
https://en.wikipedia.org/wiki/Go_Crazy_(Chris_Brown_and_Young_Thug_song)
https://en.wikipedia.org/wiki/Chris_Brown
https://en.wikipedia.org/wiki/Young_Thug
6190 / 6351 97.465% 2020 no. 40
https://en.wikipedia.org/wiki/High_Fashion_(song)
https://en.wikipedia.org/wiki/Roddy_Ricch
https://en.wikipedia.org/wiki/Mustard_(re

6237 / 6351 98.205% 2020 no. 87
https://en.wikipedia.org/wiki/Juicy_(Doja_Cat_song)
https://en.wikipedia.org/wiki/Doja_Cat
https://en.wikipedia.org/wiki/Tyga
6238 / 6351 98.221% 2020 no. 88
https://en.wikipedia.org/wiki/Be_Like_That_(Kane_Brown,_Swae_Lee_and_Khalid_song)
https://en.wikipedia.org/wiki/Kane_Brown
https://en.wikipedia.org/wiki/Swae_Lee
https://en.wikipedia.org/wiki/Khalid_(singer)
6239 / 6351 98.236% 2020 no. 89
https://en.wikipedia.org/wiki/Homesick_(Kane_Brown_song)
https://en.wikipedia.org/wiki/Kane_Brown
6240 / 6351 98.252% 2020 no. 90
https://en.wikipedia.org/wiki/Rags2Riches_(song)
https://en.wikipedia.org/wiki/Rod_Wave
6241 / 6351 98.268% 2020 no. 91
https://en.wikipedia.org/wiki/Bluebird_(Miranda_Lambert_song)
https://en.wikipedia.org/wiki/Miranda_Lambert
6242 / 6351 98.284% 2020 no. 92
https://en.wikipedia.org/wiki/Wishing_Well_(Juice_Wrld_song)
https://en.wikipedia.org/wiki/Juice_Wrld
6243 / 6351 98.299% 2020 no. 93
https://en.wikipedia.org/wiki/Does_to_Me
https

https://en.wikipedia.org/wiki/Olivia_Rodrigo
6289 / 6351 99.024% 2021 no. 39
https://en.wikipedia.org/wiki/Back_in_Blood_(song)
https://en.wikipedia.org/wiki/Pooh_Shiesty
https://en.wikipedia.org/wiki/Lil_Durk
6290 / 6351 99.04% 2021 no. 40
https://en.wikipedia.org/wiki/I_Hope_(Gabby_Barrett_song)
https://en.wikipedia.org/wiki/Gabby_Barrett
https://en.wikipedia.org/wiki/Charlie_Puth
6291 / 6351 99.055% 2021 no. 41
https://en.wikipedia.org/wiki/Dynamite_(BTS_song)
https://en.wikipedia.org/wiki/BTS
https://en.wikipedia.org//wiki/Kim_Seok-jin
https://en.wikipedia.org//wiki/Suga_(rapper)
https://en.wikipedia.org//wiki/J-Hope
https://en.wikipedia.org//wiki/RM_(rapper)
https://en.wikipedia.org//wiki/Jimin_(singer,_born_1995)
https://en.wikipedia.org//wiki/V_(singer)
https://en.wikipedia.org//wiki/Jungkook
6292 / 6351 99.071% 2021 no. 42
https://en.wikipedia.org/wiki/Wockesha
https://en.wikipedia.org/wiki/Moneybagg_Yo
6293 / 6351 99.087% 2021 no. 43
https://en.wikipedia.org/wiki/You_Right
htt

https://en.wikipedia.org/wiki/Blanco_Brown
6339 / 6351 99.811% 2021 no. 89
https://en.wikipedia.org/wiki/Take_My_Breath
https://en.wikipedia.org/wiki/The_Weeknd
6340 / 6351 99.827% 2021 no. 90
https://en.wikipedia.org/wiki/We%27re_Good
https://en.wikipedia.org/wiki/Dua_Lipa
6341 / 6351 99.843% 2021 no. 91
https://en.wikipedia.org/wiki/Hell_of_a_View
https://en.wikipedia.org/wiki/Eric_Church
6342 / 6351 99.858% 2021 no. 92
https://en.wikipedia.org/wiki/Rockin%27_Around_the_Christmas_Tree
https://en.wikipedia.org/wiki/Brenda_Lee
6343 / 6351 99.874% 2021 no. 93
https://en.wikipedia.org/wiki/Put_Your_Records_On#Ritt_Momney_version
https://en.wikipedia.org/wiki/Ritt_Momney
6344 / 6351 99.89% 2021 no. 94
https://en.wikipedia.org/wiki/Happier_Than_Ever_(song)
https://en.wikipedia.org/wiki/Billie_Eilish
6345 / 6351 99.906% 2021 no. 95
https://en.wikipedia.org/wiki/Single_Saturday_Night
https://en.wikipedia.org/wiki/Cole_Swindell
6346 / 6351 99.921% 2021 no. 96
https://en.wikipedia.org/wiki/Thi